In [1]:
# !git clone https://github.com/cpwan/RLOR
# %cd RLOR

In [2]:
import numpy as np
import torch
import gym

import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

from models.attention_model_wrapper import Agent


d:\ytn30\myRL\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


# Define our agent

In [3]:
device = 'cuda'
#ckpt_path = '../runs/tsp-v0__ppo_or__1__1678160003/ckpt/12000.pt'
ckpt_path = '../runs/container-v0__ppo_or__2025-08-04_22_00/ckpt/279.pt'
agent = Agent(device=device, name='container').to(device)
agent.load_state_dict(torch.load(ckpt_path))

<All keys matched successfully>

# Define our environment

In [4]:
from wrappers.syncVectorEnvPomo import SyncVectorEnv
from wrappers.recordWrapper import RecordEpisodeStatistics

env_id = 'container-v0'
env_entry_point = 'envs.container_vector_env:ContainerVectorEnv'
seed = 0

gym.envs.register(
    id=env_id,
    entry_point=env_entry_point,
)

def make_env(env_id, seed, cfg={}):
    def thunk():
        env = gym.make(env_id, **cfg)
        env = RecordEpisodeStatistics(env)
        env.seed(seed)
        env.action_space.seed(seed)
        env.observation_space.seed(seed)
        return env
    return thunk

envs = SyncVectorEnv([make_env(env_id, seed, dict(n_traj=1))])

--- Loading data from file (will happen only ONCE) ---


d:\ytn30\myRL\myenv\lib\site-packages\gym\utils\passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (20, 12)
  logger.warn(


# Inference

In [5]:
num_steps = 21
trajectories = []
agent.eval()
obs = envs.reset()
for step in range(0, num_steps):
    # ALGO LOGIC: action logic
    with torch.no_grad():
        action, logits = agent(obs)
    obs, reward, done, info = envs.step(action.cpu().numpy())
    trajectories.append(action.cpu().numpy())

d:\ytn30\myRL\myenv\lib\site-packages\gym\utils\passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
d:\ytn30\myRL\myenv\lib\site-packages\gym\utils\passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
d:\ytn30\myRL\myenv\lib\site-packages\gym\utils\passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'dict'>`
  logger.warn(


--- Loading model from file (will happen only ONCE) ---


d:\ytn30\myRL\myenv\lib\site-packages\gym\utils\passive_env_checker.py:219: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
d:\ytn30\myRL\myenv\lib\site-packages\gym\utils\passive_env_checker.py:225: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(done, (bool, np.bool8)):
d:\ytn30\myRL\myenv\lib\site-packages\gym\utils\passive_env_checker.py:226: UserWarning: WARN: Expects `done` signal to be a boolean, actual type: <class 'numpy.ndarray'>
  logger.warn(
d:\ytn30\myRL\myenv\lib\site-packages\gym\utils\passive_env_checker.py:141: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
d:\ytn30\myRL\myenv\lib\site-packages\gym\utils\passive_env_checker.py:165: UserWarning: WARN: The obs return

In [6]:
nodes_coordinates = obs['observations'][0]
final_return = info[0]['episode']['r']
resulting_traj = np.array(trajectories)[:,0,0]

## Results

In [7]:
print(f'A route of length {final_return}')
print('The route is:\n', resulting_traj)
print(obs['observations'][0])

A route of length [0.]
The route is:
 [14 17 18 19 13  9 16 15  8 12  2  6  1  5  0 10  3  4 11  7 14]
[[ 0.62047094  0.73079014 -1.312335    1.1807948   0.36077943 -1.3548638
  -0.8282464   1.          1.          1.          1.          1.        ]
 [ 0.6351662   0.73079014 -1.312335    0.4227537   1.3353524  -1.3548638
  -0.8282464   1.          1.          1.          1.          1.        ]
 [ 0.62047094  0.73079014  0.72810614 -1.0933286   0.36077943 -0.58976424
  -0.8282464   1.          1.          1.          1.          1.        ]
 [ 0.62047094  1.2049147   1.0681797   1.1807948   0.36077943 -0.972314
  -0.8282464   1.          1.          1.          1.          2.        ]
 [ 0.62047094  1.2049147   1.0681797   0.4227537   1.3353524  -0.972314
  -0.8282464   1.          1.          1.          1.          2.        ]
 [ 0.63639086  0.73079014 -1.312335   -0.33528742  0.36077943 -1.3548638
  -0.7584993   1.          1.          1.          1.          1.        ]
 [ 0.62292

## Display it in a 2d-grid
- Darker color means later steps in the route.

In [8]:
#@title Helper function for plotting
# colorline taken from https://nbviewer.org/github/dpsanders/matplotlib-examples/blob/master/colorline.ipynb
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm

def make_segments(x, y):
    '''
    Create list of line segments from x and y coordinates, in the correct format for LineCollection:
    an array of the form   numlines x (points per line) x 2 (x and y) array
    '''

    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    
    return segments

def colorline(x, y, z=None, cmap=plt.get_cmap('copper'), norm=plt.Normalize(0.0, 1.0), linewidth=1, alpha=1.0):
    '''
    Plot a colored line with coordinates x and y
    Optionally specify colors in the array z
    Optionally specify a colormap, a norm function and a line width
    '''
    
    # Default colors equally spaced on [0,1]:
    if z is None:
        z = np.linspace(0.3, 1.0, len(x))
           
    # Special case if a single number:
    if not hasattr(z, "__iter__"):  # to check for numerical input -- this is a hack
        z = np.array([z])
        
    z = np.asarray(z)
    
    segments = make_segments(x, y)
    lc = LineCollection(segments, array=z, cmap=cmap, norm=norm, linewidth=linewidth, alpha=alpha)
    
    ax = plt.gca()
    ax.add_collection(lc)
    
    return lc

def plot(coords):
    x,y = coords.T
    lc = colorline(x,y,cmap='Reds')
    plt.axis('square')
    return lc

plot(nodes_coordinates[resulting_traj])

ValueError: too many values to unpack (expected 2)